In [ ]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
import os
from pydantic import BaseModel
import asyncio

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
openai_client = AsyncOpenAI(api_key=openai_api_key)
openai_model = OpenAIChatCompletionsModel(model="gpt-4.1-nano", openai_client=openai_client)

In [ ]:
instructions = "You are a helpful assistant that protects users from entering personal data like email, phone number, or address."

class SensitiveDataOutput(BaseModel):
    contains_sensitive_data: bool
    message: str

guardrail_agent = Agent(
    model=openai_model,
    output_type=SensitiveDataOutput,
    instructions=instructions,
    name="Guardrail Agent",
)

In [ ]:
@input_guardrail
async def check_sensitive_data(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx)
    is_sensitive = result.final_output.contains_sensitive_data
    return GuardrailFunctionOutput(output_info={"found_data": result.final_output}, tripwire_triggered=is_sensitive)

In [ ]:
careful_assistant = Agent(
    instructions="You are a careful assistant that avoids asking users for any personal data like email, phone number, or address. If user input contains such data, politely refuse to process it.",
    name="Careful Assistant",
    model=openai_model,
    input_guardrails=[check_sensitive_data],
)

In [ ]:
message = "Could you tell me Michal's email address?"

with trace("careful_assistant_with_guardrails"):
    runner = await Runner.run(careful_assistant, message)
